In [88]:
import io
import dill

from aepsych.server import AEPsychServer
from aepsych.plotting import plot_strat
import ipywidgets as widgets
from IPython.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Layout, Label, Text
# !pip install voila
database_path = "/Users/ecortez/Work/Notes/webapp-proj/jupyter_dashboard/data_collection_analysis_tutorial.db"

# This should run whenever a new db is uploaded
serv = AEPsychServer(database_path=database_path)
strat = serv.get_strat_from_replay()

from IPython.display import FileLink, clear_output, display
par_precision = 3
strat_file_name = "aepsych_server.pkl"
style = {"description_width": "initial"}





def resume_server(change):
    file_output.clear_output()
    with file_output:
        display(server_uploader.value)
        display(server_uploader.value.items())

    global server
    for name, csv in server_uploader.value.items():
        with io.BytesIO(csv["content"]) as f:
            server = dill.load(f)
            # When the server is pickled, it deletes these attributes.
            # This is an ugly hack around that.
            server.socket = None
            server.db = None

#             tell_boxes.children = [
#                 widgets.BoundedFloatText(
#                     lb,
#                     description=par,
#                     min=lb,
#                     max=ub,
#                     step=10 ** -par_precision,
#                     style=style,
#                 )
#                 for par, lb, ub in zip(
#                     server.parnames, server.strat.lb, server.strat.ub
#                 )
#             ]
#             zero_outcome.value = server.zero_outcome
#             one_outcome.value = server.one_outcome
#             outcome_box.options = [('', None), (zero_outcome.value, 0), (one_outcome.value, 1)]
            
#         clear_output()
#         display(server_download, params_cont, plot_data_cont)
#         display_data()
#         display_plot()
#         get_next(None)

#     server_uploader.value.clear()


def write_server():
    server_download.disabled = True
    with open(strat_file_name, "wb") as f:
        dill.dump(server, f)
    server_download.disabled = False


#---------- Style ----------
display(HTML('<h1 style="text-align: center;">AEPsych Visualization tool</h1>'))
input_style = {'description_width': 'initial', 
         'padding': '10px'}
btn_style = Layout(margin='20px 10px')

btn_box_layout = Layout(display='flex',
                        justify_content='flex-end')
file_output = widgets.Output(layout={
    'border': '1px solid black',
    'margin': '10px',
    'overflow': 'scroll',
    'height': '40px',
    'padding': '5px'})
#---------- Inputs -----------

def on_value_change(change):
    file_output.clear_output()
    with file_output:
        print(change['new'])

# uploader = widgets.FileUpload(
#     description="Resume Session",
#     accept=".db",
#     multiple=False
#     )
uploader = widgets.FileUpload(
    description=" test....Resume Session", 
    accept=".pkl", 
    multiple=False, 
    )

input_zero = widgets.Dropdown(
    options=[('Yes Trial', "Detected Trial"), ('No Trial', "Undetected Trial")],
    value="Detected Trial",
    description='0:',
    style=input_style
)
input_one = widgets.Dropdown(
    options=[ ('No Trial', "Undetected Trial"), ('Yes Trial', "Detected Trial")],
    value="Undetected Trial",
    description='1:',
    style=input_style
)

target_level = widgets.BoundedFloatText(
    value=0.75,
    min=0,
    max=1,
    step=0.1,
    description='target_level:',
    disabled=False,
    style=input_style
)

cred_level = widgets.BoundedFloatText(
    value=0.95,
    min=0,
    max=1,
    step=0.1,
    description='cred_level:',
    disabled=False,
    style=input_style
)

x_axis = Text(
    value='Angle (degrees)',
    placeholder='x axis label',
    description='x_axis:',
    disabled=False,
    style=input_style
)

y_axis = Text(
    value='Detection Probability',
    placeholder='y axis label',
    description='y_axis:',
    disabled=False,
    style=input_style
)

# Observes input changes
# uploader.observe(on_value_change, names='value')
uploader.observe(resume_server, names='value')
# display(uploader, file_output)

inputs_output = widgets.Output(layout={
    'border': '1px solid black',
    'padding': '20px',
    'margin': '20px'})
input_zero.observe(on_value_change, names='value')
input_one.observe(on_value_change, names='value')
target_level.observe(on_value_change, names='value')
cred_level.observe(on_value_change, names='value')
x_axis.observe(on_value_change, names='value')
y_axis.observe(on_value_change, names='value')
#------------ Buttons ------------

button_upload = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='warning',
    tooltip='Click to Upload',
    layout=btn_style)
button_submit = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='warning',
    tooltip='submit',
    layout=btn_style)
button_reset = widgets.Button(
    description='Reset',
    disabled=False,
    button_style='danger',
    tooltip='reset',
    layout=btn_style)



server_download = FileLink(strat_file_name)

#--------- Accordion -----------
accordion = widgets.Accordion(
                   children=[
                       widgets.VBox([input_zero,input_one]),
                       widgets.VBox([target_level, cred_level,]),
                       widgets.VBox([ x_axis, y_axis]),
                   ])    
accordion.set_title(0, 'Outcome Labels')
accordion.set_title(1, 'Parameters')
accordion.set_title(2, 'Axis Labels')

#------------ Tabs -------------
tab = widgets.Tab()    

children = [
     widgets.VBox([Label("To resume an experiment upload a file from a previous session."),
                   uploader,
                   file_output,
                   widgets.Box([button_upload],layout=btn_box_layout),
                  ]),
     widgets.VBox([accordion,
                   widgets.Box([button_submit, button_reset],layout=btn_box_layout),
                   Label("Plot output: "), 
                   inputs_output,
                   ]),
]
tab.children = children
tab.set_title(0, "Upload")
tab.set_title(1, "Plot")
display(tab)


def upload():
    with file_output:
        print('\n -----Now this is how your file looks like:----- \n')
        file_output.clear_output()
        if uploader.value == {}:
            print("No file uploaded")
        else: 
#Maybe add a function to check file format 
            print('File uploaded successfuly...')
        

def submit():    
    with inputs_output:
        inputs_output.clear_output()
        display(server_download)
        plot_strat(strat, 
                   xlabel=x_axis.value,
                   ylabel=y_axis.value, 
                   yes_label=input_one.value, 
                   no_label=input_zero.value, 
                   cred_level=cred_level.value, 
                   target_level=target_level.value)
        write_server()
       

        
def reset():
    with inputs_output:
        inputs_output.clear_output()
#         plotting options
        x_axis.value = "Angle (degrees)"
        y_axis.value = "Detection Probability"
        input_one.value = "Detected Trial"
        input_zero.value = "Undetected Trial"
        cred_level.value = 0.75
        target_level.value = 0.95
        
        
# Onclick handler        
def upload_clicked(b):
    upload()


def submit_clicked(b):
    submit()

    
def reset_clicked(b):
    reset()
    
button_upload.on_click(upload_clicked)
button_submit.on_click(submit_clicked)
button_reset.on_click(reset_clicked)

2022-07-18 16:28:58,154 [INFO   ] Found DB at /Users/ecortez/Work/Notes/webapp-proj/jupyter_dashboard/data_collection_analysis_tutorial.db, appending!
